# 원본 이미지 레이블 정렬화

In [ ]:
import os
import shutil
import sqlite3

db = sqlite3.connect("../dataset.sqlite")
sql = db.cursor()


lists = os.listdir("../dataset_dir/13.한국어글자체/original/")
lists = sorted(lists, key=lambda boxes: boxes[:-4])


imageText = None


for imageInfo in tqdm(lists):
    try:
        # SQL 구문 검색
        sql.execute(f"""SELECT text FROM `handwriting_dataset` WHERE `image_id` = '{imageInfo[:-4]}'""")
        imageText = sql.fetchone()
    

        # 레이블 디렉토리 생성
        try:
            os.mkdir(f"../dataset_dir/13.한국어글자체/image/{imageText[0]}") # 디렉토리 생성
        except FileExistsError:
            pass


        # 파일 복사
        src = f"../dataset_dir/13.한국어글자체/original/"
        dir = f"../dataset_dir/13.한국어글자체/image/{imageText[0]}/"
        shutil.copyfile(src + imageInfo, dir + imageInfo) # 파일 복사

    except TypeError:
        print(imageInfo, imageText)


# 각 레이블 저장 및 전체 묶음 저장

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/image") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬


for dir in tqdm(dirList):
    fileList = os.listdir(f"../dataset_dir/13.한국어글자체/image/{dir}/") # 디랙토리 파일 리스트

    # 이미지 업스케일링 파일 저장 디랙토리 생성
    try:
        os.mkdir(f"../dataset_dir/13.한국어글자체/new_image/{dir}") # 디렉토리 생성
    except FileExistsError:
        pass

    for file in fileList:
        imageData = Image.open(f"""../dataset_dir/13.한국어글자체/image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((200, 200)) # 이미지 업스케일링

        imageData.save(f"""../dataset_dir/13.한국어글자체/new_image/{dir}/{file}""") # 이미지 업스케일링 이미지 저장

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/image") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

for dir in tqdm(dirList):
    fileList = os.listdir(f"../dataset_dir/13.한국어글자체/image/{dir}/") # 디랙토리 파일 리스트

    # npy 파일 저장 디랙토리 생성
    try:
        os.mkdir(f"../dataset_dir/13.한국어글자체/image_npy/{dir}") # 디렉토리 생성
    except FileExistsError:
        pass

    dir_dataset_tensor = [] # 데이터셋 레이블
    dir_label_tensor = [] # 데이터셋 레이블

    for file in fileList:
        tensor_label = dict.fromkeys(dirList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성
        tensor_label[dir[:-4]] = 1 # 사용자가 지정한 str 값을 1로 변경
        tensor_label = list(tensor_label.values())

        dir_label_tensor.append([tensor_label]) # 레이블 2차원 row 추가

        imageData = Image.open(f"""../dataset_dir/13.한국어글자체/image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((200, 200)) # 이미지 업스케일링

        file_tensor = np.array(imageData) # 이미지 tensor 변환
        file_tensor = file_tensor.reshape(-1) # tensor 1차원 변환

        dir_dataset_tensor.append(file_tensor.tolist()) # 레이블 데이터셋 2차원 row 추가

        np.save(f"""../dataset_dir/13.한국어글자체/image_npy/{dir}/{file[:-4]}_data""", file_tensor) # 이미지 데이터셋 파일 .npy 저장
    np.save(f"""../dataset_dir/13.한국어글자체/image_npy/{dir}/{file[:-4]}_label""", dir_label_tensor) # 이미지 레이블 파일 .npy 저장

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/image") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

dir_num = 1

for dir in tqdm(dirList):
    fileList = os.listdir(f"../dataset_dir/13.한국어글자체/image/{dir}/") # 디랙토리 파일 리스트

    # 레이블 npy 파일 저장 디랙토리 생성
    try:
        os.mkdir(f"../dataset_dir/13.한국어글자체/label_npy/{dir}") # 디렉토리 생성
    except FileExistsError:
        pass

    dir_dataset_tensor = np.array([]) # 데이터셋 레이블

    for file in fileList:
        imageData = Image.open(f"""../dataset_dir/13.한국어글자체/image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((200, 200)) # 이미지 업스케일링

        file_tensor = np.array(imageData) # 이미지 tensor 변환
        file_tensor = file_tensor.reshape(-1) # tensor 1차원 변환

        dir_dataset_tensor = np.r_[dir_dataset_tensor, file_tensor] # 레이블 데이터셋 2차원 row 추가
    
    np.save(f"../dataset_dir/13.한국어글자체/label_npy/{dir}/{dir_num}_{dir}", dir_dataset_tensor) # 레이블 데이터셋 파일 .npy 저장
    
    dir_num += 1

In [ ]:
import os
import numpy as np
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/label_npy") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

dir_num = 1
all_dataset_tensor = np.array([]) # 데이터셋 레이블

for dir in tqdm(dirList):
    dir_tensor = np.load(f"../dataset_dir/13.한국어글자체/label_npy/{dir}")
    dir_tensor.tolist()
    dir_dataset_tensor = np.r_[dir_dataset_tensor, dir_tensor] # 레이블 데이터셋 2차원 row 추가

np.save(f"../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_데이터", all_dataset_tensor) # 레이블 데이터셋 파일 .npy 저장

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/image") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

dir_num = 1

for dir in tqdm(dirList):
    fileList = os.listdir(f"../dataset_dir/13.한국어글자체/image/{dir}/") # 디랙토리 파일 리스트

    dir_dataset_tensor = [] # 데이터셋 레이블

    for file in fileList:
        imageData = Image.open(f"""../dataset_dir/13.한국어글자체/image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((200, 200)) # 이미지 업스케일링

        file_tensor = np.array(imageData) # 이미지 tensor 변환
        file_tensor = file_tensor.reshape(-1) # tensor 1차원 변환

        dir_dataset_tensor.append(file_tensor.tolist()) # 레이블 데이터셋 2차원 row 추가
    
    dir_dataset_tensor = np.array(dir_dataset_tensor)
    np.save(f"../dataset_dir/13.한국어글자체/label_npy/{dir}", dir_dataset_tensor) # 레이블 데이터셋 파일 .npy 저장
    
    dir_num += 1

In [63]:
import os
import numpy as np
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/13.한국어글자체/label_npy") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

dirList.remove("가.npy")

dir_num = 1
all_label_tensor = []


for dir in tqdm(dirList):
    tensor_label = dict.fromkeys(dirList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성
    tensor_label[dir[:-4]] = 1 # 사용자가 지정한 str 값을 1로 변경
    tensor_label = list(tensor_label.values())

    all_label_tensor.append([tensor_label]) # 레이블 2차원 row 추가
    
    try:
        all_dataset_tensor = np.load(f"../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_데이터")
    except:
        all_dataset_tensor = np.load("../dataset_dir/13.한국어글자체/label_npy/가.npy") # 레이블

    dir_tensor = np.load(f"../dataset_dir/13.한국어글자체/label_npy/{dir}")
    all_dataset_tensor = np.concatenate((all_dataset_tensor, dir_tensor), axis=0) # 데이터셋 2차원 row 추가

    np.save(f"../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_데이터", all_dataset_tensor) # 레이블 데이터셋 파일 .npy 저장
np.save(f"../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_레이블", all_label_tensor) # 레이블 데이터셋 파일 .npy 저장

  0%|          | 0/11171 [00:00<?, ?it/s]

100%|██████████| 11171/11171 [09:22<00:00, 19.87it/s]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold

model = LogisticRegression()
kfold = KFold(n_splits=3, shuffle=True, random_state=777)
scores = cross_val_score(model, X, Y, cv=kfold)

print("cv set score :", scores, "\ncv mean score :", scores.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit

model = LogisticRegression()
shuffle_split = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=3, random_state=777)

scores = cross_val_score(model, X, Y, cv=shuffle_split)

print("cv set score :", scores, "\ncv mean score :", scores.mean())

In [64]:
import numpy as np

dataset = np.load("../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_데이터.npy")
label = np.load("../dataset_dir/13.한국어글자체/label_npy/0_한국어_손글씨_레이블.npy")

print(f"데이터셋 : {dataset.shape}")
print(f"레이블 : {label.shape}")

데이터셋 : (101, 40000)
레이블 : (11171, 1, 11172)


In [57]:
import numpy as np

npy1 = np.load("../dataset_dir/13.한국어글자체/label_npy/가.npy")
npy2 = np.load("../dataset_dir/13.한국어글자체/label_npy/나.npy")

print(npy1.shape)
print(npy2.shape)

(39, 40000)
(67, 40000)


In [6]:
from PIL import Image
import numpy as np

imageData = Image.open("../dataset_dir/image/가/00004947.png").convert("L")
imageData = imageData.resize((200, 200)) # 이미지 업스케일링

file_tensor = np.array(imageData) # 이미지 tensor 변환

print(file_tensor.shape)

(200, 200)


In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

dirList = os.listdir("../dataset_dir/image") # 이미지 레이어 리스트
dirList = sorted(dirList) # 가나다 순으로 정렬

for dir in tqdm(dirList):
    fileList = os.listdir(f"../dataset_dir/image/{dir}/") # 디랙토리 파일 리스트

    # npy 파일 저장 디랙토리 생성
    try:
        os.mkdir(f"../dataset_dir/image_npy/{dir}") # 디렉토리 생성
    except FileExistsError:
        pass

    dir_dataset_tensor = [] # 데이터셋 레이블
    dir_label_tensor = [] # 데이터셋 레이블

    for file in fileList:
        # tensor_label = dict.fromkeys(dirList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성
        # tensor_label[dir[:-4]] = 1 # 사용자가 지정한 str 값을 1로 변경
        # tensor_label = list(tensor_label.values())

        # dir_label_tensor.append([tensor_label]) # 레이블 2차원 row 추가

        imageData = Image.open(f"""../dataset_dir/image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((200, 200)) # 이미지 업스케일링

        file_tensor = np.array(imageData) # 이미지 tensor 변환

        dir_dataset_tensor.append(file_tensor.tolist()) # 레이블 데이터셋 2차원 row 추가

        np.save(f"""../dataset_dir/image_npy/{dir}/{file[:-4]}_data""", file_tensor) # 이미지 데이터셋 파일 .npy 저장

100%|██████████| 11172/11172 [18:04<00:00, 10.30it/s]


In [66]:
arr_concat_v.shape

(106, 40000)